# Import Libraries

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


# Fetch the Steam Search Page

In [10]:
url = "https://store.steampowered.com/search/"
params = {"term": "games", "supportedlang": "english", "ndl": 1, "start": 0}
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, params=params, headers=headers)
print(response.status_code)  



200


# Parse HTML

In [11]:
soup = BeautifulSoup(response.text, "html.parser")
rows = soup.select(".search_result_row")
print("Games found:", len(rows))


Games found: 50


# Extract One Game’s Info

In [12]:
row = rows[0]

title = row.select_one(".title").get_text(strip=True)
release_date = row.select_one(".search_released").get_text(strip=True)

print("Title:", title)
print("Release Date:", release_date)


Title: Games
Release Date: Coming soon


# Add Price & Discount

In [13]:
price = row.select_one(".discount_final_price")
orig_price = row.select_one(".search_price strike")
discount = row.select_one(".search_discount span")

print("Price:", price.get_text(strip=True) if price else None)
print("Original Price:", orig_price.get_text(strip=True) if orig_price else None)
print("Discount:", discount.get_text(strip=True) if discount else "0%")


Price: Free
Original Price: None
Discount: 0%


# Platforms 

In [14]:
platforms = []
if row.select_one(".platform_img.win"): platforms.append("Windows")
if row.select_one(".platform_img.mac"): platforms.append("Mac")
if row.select_one(".platform_img.linux"): platforms.append("Linux")

print("Platforms:", ", ".join(platforms))


Platforms: Windows


# Reviews

In [15]:
review_tag = row.select_one(".search_review_summary")
if review_tag and review_tag.has_attr("data-tooltip-html"):
    tooltip = review_tag["data-tooltip-html"]
    review_rating = tooltip.split("<br>")[0]
    review_count = re.sub(r"[^\d]", "", tooltip.split("<br>")[-1])
else:
    review_rating = None
    review_count = None

print("Review Rating:", review_rating)
print("Review Count:", review_count)


Review Rating: None
Review Count: None


# Loop Over Games

In [16]:
games = []

for row in rows:
    title = row.select_one(".title").get_text(strip=True) if row.select_one(".title") else None
    release_date = row.select_one(".search_released").get_text(strip=True) if row.select_one(".search_released") else None
    price = row.select_one(".discount_final_price")
    orig_price = row.select_one(".search_price strike")
    discount = row.select_one(".search_discount span")

    platforms = []
    if row.select_one(".platform_img.win"): platforms.append("Windows")
    if row.select_one(".platform_img.mac"): platforms.append("Mac")
    if row.select_one(".platform_img.linux"): platforms.append("Linux")

    review_tag = row.select_one(".search_review_summary")
    if review_tag and review_tag.has_attr("data-tooltip-html"):
        tooltip = review_tag["data-tooltip-html"]
        review_rating = tooltip.split("<br>")[0]
        review_count = re.sub(r"[^\d]", "", tooltip.split("<br>")[-1])
    else:
        review_rating = None
        review_count = None

    games.append({
        "title": title,
        "price": price.get_text(strip=True) if price else None,
        "original_price": orig_price.get_text(strip=True) if orig_price else None,
        "discount": discount.get_text(strip=True) if discount else "0%",
        "release_date": release_date,
        "platforms": ", ".join(platforms),
        "review_rating": review_rating,
        "review_count": review_count
    })

df = pd.DataFrame(games)
df.head()


,title,price,original_price,discount,release_date,platforms,review_rating,review_count
0,Games,Free,None,0%,Coming soon,Windows,None,None
1,Sultan's Game,₹935.00,None,0%,"30 Mar, 2025","Windows, Mac",Very Positive,
2,Counter-Strike 2,Free,None,0%,"21 Aug, 2012","Windows, Linux",Very Positive,
3,PEAK,₹359.00,None,0%,"16 Jun, 2025",Windows,Very Positive,
4,PUBG: BATTLEGROUNDS,Free,None,0%,"21 Dec, 2017",Windows,Mixed,


# Save to CSV

In [18]:
df = pd.DataFrame(all_games)
df.to_csv("steam_games1000.csv", index=False, encoding="utf-8-sig")
print("Data saved as steam_games.csv")


Data saved as steam_games.csv
